In [9]:
import csv
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split

In [ ]:
# Load data
X = pd.read_csv('train_values.csv')[:28955]
Y = pd.read_csv('train_labels.csv')[:28955]
print("len Y:",len(Y))
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Convert y_train and y_test to 1-dimensional arrays
y_train = y_train.values[:, 1]  # Select the first column
y_test = y_test.values[:, 1]    # Select the first column

len X: 86867
len Y: 28955


ValueError: Found input variables with inconsistent numbers of samples: [86867, 28955]

In [6]:
clf = LazyClassifier(verbose=1, ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

  0%|          | 0/32 [00:00<?, ?it/s]

  3%|▎         | 1/32 [00:10<05:16, 10.22s/it]

{'Model': 'AdaBoostClassifier', 'Accuracy': 0.6291805094130676, 'Balanced Accuracy': np.float64(0.4881383439411125), 'ROC AUC': None, 'F1 Score': 0.589830892888736, 'Time taken': 10.224402666091919}


  6%|▋         | 2/32 [00:22<05:42, 11.41s/it]

{'Model': 'BaggingClassifier', 'Accuracy': 0.7046788482834995, 'Balanced Accuracy': np.float64(0.6367530441679269), 'ROC AUC': None, 'F1 Score': 0.6997972071959867, 'Time taken': 12.23862361907959}


  9%|▉         | 3/32 [00:22<03:06,  6.42s/it]

{'Model': 'BernoulliNB', 'Accuracy': 0.5192691029900333, 'Balanced Accuracy': np.float64(0.5480549768110654), 'ROC AUC': None, 'F1 Score': 0.5214987963361236, 'Time taken': 0.4833643436431885}


  9%|▉         | 3/32 [00:58<09:23, 19.42s/it]


KeyboardInterrupt: 